<a href="https://colab.research.google.com/github/nvalsangkar/Waterline-Jamuna/blob/main/WaterlineJamuna_Sentinel2_WaterDelineation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is provided as supplementary material to the manuscript Combining Earth Observations with Ground Data to Assess River Topography and Morphologic Change: Case Study of the Lower Jamuna River, submitted to International Journal of Applied Earth Observations and Geoinformation, 2024. Authors: N. Valsangkar, A. Nelson, and Md. F. Hassan.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
# Workflow for surface water detection from Sentinel-2 data was adapted from published code repositories by Boothroyd et. al (2021) https://doi.org/10.3389/fenvs.2021.657354
# Original scripts are available at: https://code.earthengine.google.com/1d1cc675904221567886f2e91b21d87d
# Script was translated to Python (Colab) and modified to use Sentinel-2 data, rather than Landsat data as used in the original study

In [ ]:
# Import libraries

#!pip install ipyleaflet==0.18.2
#%pip install -U geemap

import ee
import geemap
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project='YOUR PROJECT HERE')

In [ ]:
# Define region of interest (roi): either use the area provided, or uncomment the code below and draw a polygon on the map (rename the polygon 'roi').
Map=geemap.Map()
roi = ee.Geometry.Rectangle(89.6206, 24.4842, 89.85,  23.769174)
Map.centerObject(roi,8)
Map

In [ ]:
# Define time filter (change the start and end dates):
year=2017
year0=2017
yeartext=str(year)
year0text=str(year0)
startdate="-11-05"
enddate="-11-07"
sDate_T1 = year0text+startdate
eDate_T1 = yeartext+enddate
cloud_cover = 10 # Upper limit of cloud cover % for filtering

# Optionally modify the classification parameters:
#The Modified Normalized Difference Water Index (mNDWI), Normalized Difference Vegetation Index (NDVI), and Enhanced Vegetation Index (EVI) were calculated for water detection using the equations below:
# mNDWI = (𝜌𝑔𝑟𝑒𝑒𝑛−𝜌𝑆𝑊𝐼𝑅1) /(𝜌𝑔𝑟𝑒𝑒𝑛+𝜌𝑆𝑊𝐼𝑅1)
# NDVI = (𝜌𝑁𝐼𝑅−𝜌𝑅𝑒𝑑)/(𝜌𝑁𝐼𝑅+𝜌𝑅𝑒𝑑)
# EVI = 2.5 × (𝜌𝑁𝐼𝑅−𝜌𝑅𝑒𝑑)/(1+𝜌𝑁𝐼𝑅+6𝜌𝑅𝑒𝑑−7.5𝜌𝑏𝑙𝑢𝑒)
# Where ρblue, ρgreen, ρred, ρNIR, and ρSWIR1 are the surface reflectance values of Landsat blue band (0.45–0.52), green band (0.52–0.60), red band (0.63–0.69), near-infrared band (0.77–0.90), and shortwave infrared band (1.55–1.75) μm, respectively.
# Zou et al (2018) classify water where ((mNDWI > EVI or mNDWI > NDVI) and EVI < 0.1)
# (Berdoldi et al 2011 use the following NDVI thresholds: gravel and water=NDVI<0.1, sparse vegetation=NDVI0.1-0.2, dense vegetation=NDVI>0.2)
mndwi_param = -0.40;
ndvi_param = 0.10;
cleaning_pixels = 100;
ds_veg_min = 0.20

In [14]:
# Standardise band names for S2 data:
bn1 = ["B2", "B3", "B4", "B11", "B8", "B12","QA60"]
bns = ["Blue", "Green", "Red", "Swir1", "Nir", "Swir2","QA60"]
S2 = (ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
  .select(bn1, bns)
  .filterDate(sDate_T1, eDate_T1)
  .filterBounds(roi)
  .map(lambda image: image.clip(roi))
  .map(lambda img: img.set('date', img.date().format('YYYY-MM-dd')))
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover))
)

count_s2 = S2.size()
dates_s2 = S2.distinct('date').aggregate_array('date')
print(str(count_s2.getInfo()),': Number of Sentinel-2 tiles')
print(str(dates_s2.getInfo()),': Acquisition Dates')

# Set functions for classification:
def Ndvi(image):
  # calculate normalized difference vegetation index
  ndvi = image.normalizedDifference(["Nir", "Red"]).rename("ndvi")
  return(ndvi)

def Lswi(image):
  # calculate land surface water index
  lswi = image.normalizedDifference(['Nir', 'Swir1']).rename('lswi')
  return(lswi)

def Mndwi(image):
  # calculate modified normalized difference water index
  mndwi = image.normalizedDifference(['Green', 'Swir1']).rename('mndwi')
  return(mndwi)

def Evi(image):
  # calculate the enhanced vegetation index
  evi = image.expression('2.5 * (Nir - Red) / (1 + Nir + 6 * Red - 7.5 * Blue)', {
    'Nir': image.select(['Nir']),
    'Red': image.select(['Red']),
    'Blue': image.select(['Blue'])
    })
  return(evi.rename(['evi']))

# Set Visualisation parameters:
params_true = {'crs': 'EPSG:4326', 'region': roi, 'min': 0.0, 'max': 0.3, 'bands': ["Red", "Green", "Blue"], 'dimensions': 1000}
params_false = {'crs': 'EPSG:4326', 'region': roi, 'min': 0.0, 'max': 0.3, 'bands': ["Swir1", "Red", "Green"], 'dimensions': 1000}
params_waterViz = {'crs': 'EPSG:4326', 'region': roi, 'min': 0, 'max': 1, 'palette': ['white', 'blue'],'dimensions': 1000}
params_activebeltViz = {'crs': 'EPSG:4326', 'region': roi, 'min': 0, 'max': 1, 'palette': ['white', 'grey'],'dimensions': 1000}
params_activeViz = {'crs': 'EPSG:4326', 'region': roi, 'min': 0, 'max': 1, 'palette': ['white', 'grey'],'dimensions': 1000}
params_sparse_veg_Viz={'crs': 'EPSG:4326', 'region': roi, 'min': 0, 'max': 1, 'palette': ['white', 'grey'],'dimensions': 1000}

waterViz = {'min': 0, 'max': 1, 'palette': ['white', 'blue']}
activebeltViz = {'min': 0, 'max': 1, 'palette': ['white', 'grey']}
activeViz = {'min': 0, 'max': 1, 'palette': ['white', 'MediumOrchid ']}
active_cleanedViz = {'min': 0, 'max': 1, 'palette': ['white', 'OrangeRed ']}
sparse_veg_Viz = {'min': 0, 'max': 1, 'palette': ['white', 'Green ']}

# Filter date range, roi and apply simple cloud processing:
def CloudMask(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )
  return image.updateMask(mask).divide(10000)


imgCol = (S2.map(CloudMask))

# Define and rename quantiles of interest:
bnp50 = ['Blue_p50', 'Green_p50', 'Red_p50', 'Swir1_p50', 'Nir_p50', 'Swir2_p50','QA60_p50']
p50 = imgCol.reduce(ee.Reducer.percentile([50])).select(bnp50, bns)

# Apply to each percentile:
mndwi_p50 = Mndwi(p50);
ndvi_p50 = Ndvi(p50);
evi_p50 = Evi(p50);
lswi_p50 = Lswi(p50);


2 : Number of Sentinel-2 tiles
['2017-11-06'] : Acquisition Dates


In [ ]:
# Water classification from (Zou 2018):
water_p50 = (mndwi_p50.gt(ndvi_p50).Or(mndwi_p50.gt(evi_p50))).And(evi_p50.lt(0.1))
waterMasked_p50 = water_p50.updateMask(water_p50.gt(0));

# Active river belt classification:
activebelt_p50 = (mndwi_p50.gte(mndwi_param)).And(ndvi_p50.lte(ndvi_param))
activebeltMasked_p50 = activebelt_p50.updateMask(activebelt_p50.gt(0))
active_p50 = (water_p50).Or(activebelt_p50)
sparseveg_p50 = (mndwi_p50.gte(mndwi_param)).And(ndvi_p50.lte(ds_veg_min)).And(ndvi_p50.gte(ndvi_param))

# Clean binary active channel:
smooth_map_p50 = (active_p50
           .focalMode(
           10, 'octagon', 'pixels', 1
           )
           .mask(active_p50.gte(1)))

noise_removal_p50 = (active_p50
            .updateMask(active_p50.connectedPixelCount(cleaning_pixels, False).gte(cleaning_pixels))
            .unmask(smooth_map_p50))

noise_removal_p50_Masked = noise_removal_p50.updateMask(noise_removal_p50.gt(0));

# Clean sparse vegetation mask
veg_smooth_map_p50 = (sparseveg_p50
           .focalMode(
           10,'octagon', 'pixels', 1
           )
           .mask(sparseveg_p50.gte(1)))

veg_noise_removal_p50 = (sparseveg_p50
            .updateMask(sparseveg_p50.connectedPixelCount(cleaning_pixels, False).gte(cleaning_pixels))
            .unmask(veg_smooth_map_p50))

veg_noise_removal_p50_Masked = veg_noise_removal_p50.updateMask(veg_noise_removal_p50.gt(0))

# Define outputs:
True_colour = p50.select(["Red", "Green", "Blue"])
False_colour = p50.select(["Swir1", "Red", "Green"])
Wetted_channel = waterMasked_p50
Alluvial_deposits = activebeltMasked_p50
Active_channel_binary_mask = noise_removal_p50_Masked
Sparse_veg = veg_noise_removal_p50_Masked

def cloudy(image):
  return image.multiply(0.0001).clip(roi)
cloudy = (S2.filterDate(sDate_T1, eDate_T1)
                 .filterBounds(roi)
                 .map(cloudy)
                   )

In [ ]:
# Display on map:
Map
Map.addLayer(True_colour.clip(roi), params_true, 'True colour temporal composite', False)
Map.addLayer(False_colour.clip(roi), params_false, 'False colour temporal composite', True)
Map.addLayer(Alluvial_deposits, activebeltViz, 'Alluvial deposits', False)
Map.addLayer(Sparse_veg, sparse_veg_Viz, 'Sparse Vegetation', False)
Map.addLayer(Active_channel_binary_mask, activebeltViz, 'Active channel binary mask', True)
Map.addLayer(Wetted_channel, waterViz, 'Wetted channel', False)

In [ ]:
# Export to Google Drive:
task=ee.batch.Export.image.toDrive(
  image=Wetted_channel,
  description= str(sDate_T1),
  region= roi,
  scale=10,
  folder= 'GEE_Output',
  maxPixels= 1e13,
  formatOptions={
        'cloudOptimized': True
    }
)
task.start()